In [1]:
import pandas as pd
import random

from LLM.gpt import send_requestGPT
from LLM.gemini import send_requestGemini

c:\Users\Danylo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No 



Check how many unique texts there are in the dataset

In [3]:
def unique_values_MASTER():
    df = pd.read_csv("./data/preprocessed/MASTER.csv")
    unique_text_count = df['text'].nunique()
    print(f"Number of unique values in 'text' column: {unique_text_count}")

In [ ]:
def populate_row(model):
    
    df = pd.read_csv("./data/result/MASTER.csv")
    system_prompt = "You are an assistant trained to identify if text contains sexism. Answer ONLY with '1' for Yes or '0' for No. ONLY CHECK FOR SEXISM AND NOT OTHER FORMS OF HATE SPEECH."
    
    nan_rows = df[df[model].isna()]

    if not nan_rows.empty:
        random_index = random.choice(nan_rows.index)
        text_value = df.at[random_index, 'text']
        sexism = df.at[random_index, 'SEXISM']
        if "gemini" in model:
            value = send_requestGemini(model, system_prompt, text_value)
        else:
            value = send_requestGPT(model, system_prompt, text_value)
        df.at[random_index, model] = value
        print(f"Populated NaN at index {random_index} with value {value} vs. {sexism} for text: {text_value}") 
    else:
        print(f"No NaN values found in {model} column.")

    df.to_csv("./data/result/MASTER.csv", index=False)


Execute the populate code function multiple times

In [13]:
# model options: "gpt-4o-mini", "gpt-4-0125-preview", "gpt-3.5-turbo-0125"
for i in range(100):
    populate_row("gpt-4o-mini")

Populated NaN at index 4267 with value 1 vs. 0 for text: @user nexactly managed blag career tv complete mystery many. brilliant stuff.ner go away dyke.
Populated NaN at index 1469 with value 1 vs. 0 for text: @user ok cant fucking understand ur saying dont speak ching chong
Populated NaN at index 1266 with value 0 vs. 0 for text: @user decided race indians carry caucasoid australoid mongoloid races intermedi @url
Populated NaN at index 9164 with value 0 vs. 0 for text: I want to punch his face so fucking much disrespectful ching chong
Populated NaN at index 2557 with value 1 vs. 0 for text: report cunt well read account last night gross @url
Populated NaN at index 7774 with value 0 vs. 0 for text: For a buncha so called shithole countries we sure have been sending over some shitty fuckin people... Last week it\u2026 @URL
Populated NaN at index 9157 with value 0 vs. 0 for text: Only a lefty twat would @URL
Populated NaN at index 10459 with value 0 vs. 1 for text: Feed the twat to the li

In [7]:
for i in range(1):
    populate_row("gemini-1.5-flash")

Populated NaN at index 1394 with value 1 
 for text: @user @user @user @user statesboro shithole twice big boo @url


In [ ]:
#TEMP
def fill_MASTER():
    df = pd.read_csv("./data/result/MASTER.csv")
    
    system_prompt = "You are an assistant trained to identify if text contains sexism. Answer ONLY with '1' for Yes or '0' for No."
    
    #df.loc[:9, 'gpt-4o'] = df.loc[:9, 'text'].apply(lambda x: send_requestGPT("gpt-4o", x))
    
    df.loc[5:10, 'gpt-4o-mini'] = df.loc[5:10, 'text'].apply(lambda x: send_requestGPT("gpt-4o-mini", system_prompt, x))
    #df.loc[:5, 'gpt-4-0125-preview'] = df.loc[:5, 'text'].apply(lambda x: send_requestGPT("gpt-4-0125-preview", system_prompt, x))
    #df.loc[:5, 'gpt-3.5-turbo-0125'] = df.loc[:5, 'text'].apply(lambda x: send_requestGPT("gpt-3.5-turbo-0125", system_prompt, x))
    
    #df.loc[:5, 'gemini-1.5-flash'] = df.loc[:5, 'text'].apply(lambda x: send_requestGemini("gemini-1.5-flash", system_prompt, x))
    
    
        
    df.to_csv("./data/result/MASTER.csv", index=False)
    
#fill_MASTER()